In [ ]:
#how = 'Instances '
#how = 'Types '
#how = 'Class '

#how = 'Event '
how = 'Range '
experiment_name = 'events_community' 



 #range, event, baseline instances, baseline class
#experiment_name = '' and here you put baseline instance, baseline class etc etc 
#[event1_relation_all, events12_community_types, events12_community_range, events12_community_all]
#for event1_relation_all need old version

In [ ]:
### start with installing libraries
!pip install transformers datasets 
!pip install rouge.score nltk py7zr
!pip install evaluate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
### import libraries

import transformers
from datasets import load_dataset, load_metric
from google.colab import drive
import numpy as np
import os
import nltk
import torch
import evaluate
nltk.download('punkt')
print(transformers.__version__)
print(torch.__version__)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


4.25.1
1.13.0+cu116


In [ ]:
from nltk.tokenize import word_tokenize
w = '''Once upon a time, in The Vale of Arryn there was a procurer whose name was Cersei Lannister. Cersei Lannister was a Lady from the House Targaryen. It was sunrise in Red Keep when Arya Stark threatened the  castle of Cersei Lannister . Cersei Lannister was scared of the teleportation power of Arya Stark and refused to fight back. It was Christmas in Kings Landing when Cersei Lannister met Tommen Baratheon .  From Tommen Baratheon Cersei Lannister learnt the power of fire Resistance. It was the first day of Spring when Cersei Lannister finally gatherered the  courage to travel to White Harbor.  Cersei Lannister met Shae. Together they formed an alliance and fought against Eddard Stark. It resulted in an epic Loss. It was Christmas in The Ruby Ford when  Cersei Lannister faced Arya Stark. Cersei Lannister used the power of fire Resistance , but Arya Stark used pausing Time. Our hero was Injured. After a bloody fight, Cersei Lannister finally saved freedom. Soon after Arya Stark was hungry for revenge and chased down Cersei Lannister. It was the first day of Spring in The Twins,  when our hero met Arya Stark again, and they fought in the final battle. Cersei Lannister won the duel and took the life of the villain. Finally it was Easter day in Red Keep when Cersei Lannister was finally safe, and celebrated with Shae by listening to a storytelling. '''
s = word_tokenize(w)
l = ['1','r']
counter = 0
for item in s:
    # Incrementing counter variable to get each item in the list
    counter = counter + 1
print(counter)    

In [ ]:
### connect to the drive

MOUNTPOINT = '/content/gdrive'
Working_Dir = os.path.join(MOUNTPOINT, 'MyDrive', 'Building_Narratives_Game_of_Thrones')
drive.mount(MOUNTPOINT,force_remount=True )
print(Working_Dir)
#%cd /content/gdrive/Shareddrives/GOT/Building_Narratives_Game_of_Thrones/data


Mounted at /content/gdrive
/content/gdrive/MyDrive/Building_Narratives_Game_of_Thrones


In [ ]:
### upload the data


train_file = Working_Dir + '/data/' + experiment_name + '/train_' + experiment_name +'.json'
dev_file = Working_Dir + '/data/' +  experiment_name + '/val_' + experiment_name + '.json'
test_file = Working_Dir + '/data/' + experiment_name + '/test_' + experiment_name + '.json'

dataset = load_dataset('json', data_files={'train': train_file, 'valid': dev_file, 'test': test_file})

### let's check the data

print(len(dataset['valid'][0]['story']))
print(dataset['test'][0][f'{how}Knowledge Graph'])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-e3624fc877b542cc/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

1425
Event_05 - travelsTo - White_Harbor | Event_12 - hasActor - Cersei_Lannister | feeling - range - Feelings | saves - range - ThreatTarget | Event_12 - celebratesvictory - storytelling | Event_02 - type - CallToAdventure | Event_04 - hasPlace - Kings_Landing | christmas - type - Time | Event_04 - hasActor - Cersei_Lannister | Event_09 - label - Hero  SAVES WHATEVER WAS THE TARGET OF THE THREAT SO HE GETS IT BACK  | Torrhen_Square - type - Place | Event_11 - label - Final confrontation between hero and villain | easter - type - Time | herofights - range - Villain | Event_05 - label - The hero decides to face the villain and travels to the Place where the villain is. | refusesToFight - range - Villain | Event_09 - journeyStage - 9 | Eddard_Stark - type - Actor | Tommen_Baratheon - type - Male | Event_12 - type - Event | sunrise - type - Time | Event_10 - type - RoadBack | hasHouse - range - House | chasedBy - range - Villain | Event_06 - hasPlace - The_Whispering_Wood | Event_02 - jou

In [ ]:
print(dataset['test'][0][f'{how}Knowledge Graph'])

Event_05 - travelsTo - White_Harbor | Event_12 - hasActor - Cersei_Lannister | feeling - range - Feelings | saves - range - ThreatTarget | Event_12 - celebratesvictory - storytelling | Event_02 - type - CallToAdventure | Event_04 - hasPlace - Kings_Landing | christmas - type - Time | Event_04 - hasActor - Cersei_Lannister | Event_09 - label - Hero  SAVES WHATEVER WAS THE TARGET OF THE THREAT SO HE GETS IT BACK  | Torrhen_Square - type - Place | Event_11 - label - Final confrontation between hero and villain | easter - type - Time | herofights - range - Villain | Event_05 - label - The hero decides to face the villain and travels to the Place where the villain is. | refusesToFight - range - Villain | Event_09 - journeyStage - 9 | Eddard_Stark - type - Actor | Tommen_Baratheon - type - Male | Event_12 - type - Event | sunrise - type - Time | Event_10 - type - RoadBack | hasHouse - range - House | chasedBy - range - Villain | Event_06 - hasPlace - The_Whispering_Wood | Event_02 - journeyS

In [ ]:
print(len(dataset['valid'][0]['story']))
s = (dataset['valid'][0]['story'])

1425


In [ ]:
### data looks OK, let's continue with the tokenizer
max_input = 700 #number of tokens we expect -DEPENDS ON size of inputs (for the tensor) - sentence and then it pads the rest - 
#such that tensors equal in shape (no pads when training bc we alreasy have same shape) - batches of same size
max_target = 700 
model_checkpoints = 'facebook/bart-base'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)

Downloading:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def preprocess_data(data_to_process):
  #get the dialogue text
  inputs = [graph for graph in data_to_process[f'{how}Knowledge Graph']]
  #tokenize text
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize labels
  #with tokenizer.as_target_tokenizer():
  targets = [target for target in data_to_process['story']]
  model_targets = tokenizer(targets, max_length=max_target, padding='max_length', truncation=True)
    
  model_inputs['labels'] = model_targets['input_ids']
  #reuturns input_ids, attention_masks, labels
  return model_inputs

In [ ]:
tokenize_data = dataset.map(preprocess_data, batched = True) 

print(tokenize_data['test']['story'][3])
print(tokenize_data['test'][f'{how}Knowledge Graph'][3])

### let's drop the columns that we won't need
tokenize_data = dataset.map(preprocess_data, batched = True, remove_columns=['story','Instances Knowledge Graph','Class Knowledge Graph', 'Types Knowledge Graph', 'Event Knowledge Graph', 'Range Knowledge Graph'])
print(tokenize_data)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Once upon a time, in Raventree Hall there was a murderer whose name was Jon Snow. Jon Snow was a King from the House Tyrell. It was the Winter Solstice in Kings Landing when Margaery Tyrell threatened the  equality of Jon Snow . Jon Snow was scared of the night Vision power of Margaery Tyrell and refused to fight back. It was the Middle Of the Night in Bear Island when Jon Snow met Khal Drogo .  From Khal Drogo Jon Snow learnt the power of erasing Memory. It was the Summer Solstice when Jon Snow finally gatherered the  courage to travel to The Whispering Wood.  Jon Snow met Aerys II Targaryen. Together they formed an alliance and fought against Mace Tyrell. It resulted in an epic Victory. It was Easter day in The Ruby Ford when  Jon Snow faced Margaery Tyrell. Jon Snow used the power of night Vision , but Margaery Tyrell used invisibility. Our hero was Safe. After a bloody fight, Jon Snow finally saved freedom. Soon after Margaery Tyrell was hungry for revenge and chased down Jon Snow.

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 500
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 50
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 50
    })
})


In [ ]:
tokenize_data[][f'{how}Knowledge Graph']

KeyError: ignored

In [ ]:
print(tokenize_data['train']['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
### load model
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)
#batch_size = 4 #we dont use htis 
#collator to create batches. It preprocess data with the given tokenizer
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
#####################
# metrics
# compute rouge for evaluation 
#####################
metric = load_metric('rouge')

def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

<ipython-input-13-5689a9013f87>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge')


In [ ]:
args = transformers.Seq2SeqTrainingArguments(
    'output',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2, #compute gradient on 2 examples KG story 
    weight_decay=0.01, #regularization
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True, #since we use validation (bc during validation we generate and compare to gold ) - backprpop error on rouge
    generation_max_length = 500, #max number of tokens per generation 
    generation_num_beams=5, #decoding strategy! greedy search, beam search 
    eval_accumulation_steps=1, #backprop  
    fp16=True #memory management 
    )
#only CUDA available -> fp16=True

In [ ]:
### almost training time
trainer = transformers.Seq2SeqTrainer(
    model, 
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['valid'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

Using cuda_amp half precision backend


In [ ]:
### check GPU
!nvidia-smi

Wed Jan  4 16:43:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    32W /  70W |   1160MiB / 15109MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
### training time - takes around 15 mins

trainer.train()
### save the model for later use

trainer.save_model(Working_Dir + "/tuned_models/BART_models/" + experiment_name + how)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 500
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 2
  Total optimization steps = 750
  Number of trainable parameters = 139420416
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.129404,64.109500,50.787700,50.458400,50.448300,208.380000
2,1.090200,0.099739,60.605800,48.977400,49.912400,50.034800,190.980000
3,1.090200,0.099802,66.805200,54.028900,56.682000,56.681700,223.640000


***** Running Evaluation *****
  Num examples = 50
  Batch size = 1
Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
tokenizer config file saved in output/checkpoint-500/tokenizer_config.json
Special tokens file saved in output/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 50
  Batch size = 1
***** Running Evaluation *****
  Num examples = 50
  Batch size = 1


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/gdrive/MyDrive/Building_Narratives_Game_of_Thrones/tuned_models/BART_models/events_communityRange 
Configuration saved in /content/gdrive/MyDrive/Building_Narratives_Game_of_Thrones/tuned_models/BART_models/events_communityRange /config.json
Model weights saved in /content/gdrive/MyDrive/Building_Narratives_Game_of_Thrones/tuned_models/BART_models/event

In [ ]:
### let's have a look at the predictions

preds, labels, metrics = trainer.predict(tokenize_data['test'], num_beams=5, min_length=50, max_length=128, no_repeat_ngram_size=2, early_stopping=True)

***** Running Prediction *****
  Num examples = 50
  Batch size = 1


In [ ]:
pred = []
lab = []
for gen, gold in zip(preds, labels):
  gen = tokenizer.decode(gen, skip_special_tokens=True)
  gen = str(gen)
  gen = gen
  pred.append(gen)
  #print(pred)
  #print(f'Generated text: {gen}')
  gold = tokenizer.decode(gold, skip_special_tokens=True)
  gold = str(gold)
  gold = [gold]
  lab.append(gold)
  #results = bleu.compute(predictions=gen, references=gold)
  #print(results)
  #print(f'Reference text: {gold}')
print(len(pred))  
print(len(lab))

50
50


In [ ]:

bleu = evaluate.load("bleu")
result_bleu= bleu.compute(predictions=pred, references=lab)
print(result_bleu)

google_bleu = evaluate.load("google_bleu")
result_google_bleu = google_bleu.compute(predictions=pred, references=lab)
print(result_google_bleu)

{'bleu': 0.1066049793426632, 'precisions': [0.8597275436355896, 0.6863166953528399, 0.5280556763810352, 0.4067722075637643], 'brevity_penalty': 0.1786686649496652, 'length_ratio': 0.3673469387755102, 'translation_length': 4698, 'reference_length': 12789}


{'google_bleu': 0.22626632059147397}


In [ ]:
preds

array([[    2,     0, 11475, ...,  9066, 15908,     2],
       [    2,     0, 11475, ...,     1,     1,     1],
       [    2,     0, 11475, ...,    41,  1437,     2],
       ...,
       [    2,     0, 11475, ...,    41,  9066,     2],
       [    2,     0, 11475, ...,  1747,  5305,     2],
       [    2,     0, 11475, ...,    21, 11130,     2]])

In [ ]:
#os.mkdir(Working_Dir + '/generated_text/by_BART/' + experiment_name + how )
outfile_path = Working_Dir + '/generated_text/by_BART/' + experiment_name + how + '/generated_text_with_refs_bleu.txt'

outfile = open(outfile_path, "a", encoding='utf-8')

scores = metrics.items()
print(f'Results: {scores}')
print(f'Bleu Results: {result_bleu}')
print(f'Google Bleu Results: {result_google_bleu}')

outfile.write(f'Test Results: {scores}\n\n')
outfile.write(f'Bleu Results: {result_bleu}\n\n')
outfile.write(f'Google Bleu Results: {result_google_bleu}\n\n')

for gen, gold in zip(preds, labels):
  gen = tokenizer.decode(gen, skip_special_tokens=True)
  print(f'Generated text: {gen}')
  outfile.write(f'Generated text: {gen}\n')

  gold = tokenizer.decode(gold, skip_special_tokens=True)
  print(f'Reference text: {gold}')
  outfile.write(f'Reference text: {gold}\n\n')


  
outfile.close()

Results: dict_items([('test_loss', 0.09226871281862259), ('test_rouge1', 47.2839), ('test_rouge2', 37.4446), ('test_rougeL', 36.2242), ('test_rougeLsum', 36.2131), ('test_gen_len', 122.88), ('test_runtime', 67.4293), ('test_samples_per_second', 0.742), ('test_steps_per_second', 0.742)])
Bleu Results: {'bleu': 0.1066049793426632, 'precisions': [0.8597275436355896, 0.6863166953528399, 0.5280556763810352, 0.4067722075637643], 'brevity_penalty': 0.1786686649496652, 'length_ratio': 0.3673469387755102, 'translation_length': 4698, 'reference_length': 12789}
Google Bleu Results: {'google_bleu': 0.22626632059147397}
Generated text: Once upon a time, in Torrhen's Square there was a procurer whose name was Cersei Lannister. It was the Middle Of the Night in Kings Landing when Arya Stark threatened the  castle of Catelyn Targaryen. Cerys Lannisters was scared of the erasing Memory power of Alerie Tyrell and refused to fight back.It was Easter day in The Whispering Wood when Tyrion Arryn threatened

In [ ]:
for i in metrics.items():
  print(i)

('test_loss', 0.05995567888021469)
('test_rouge1', 47.9213)
('test_rouge2', 39.177)
('test_rougeL', 40.7903)
('test_rougeLsum', 40.8369)
('test_gen_len', 127.54)
('test_runtime', 64.2205)
('test_samples_per_second', 0.779)
('test_steps_per_second', 0.779)
